# Motion with Accelerometers and Gyroscopes
By Tre Tomaszewski

Uses concepts described on [Accelerometer - Wikipedia](https://en.wikipedia.org/wiki/Accelerometer)

In [3]:
%matplotlib widget

import ipywidgets as widgets
import traitlets
from IPython.display import display

from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

# Thanks to `ImportanceOfBeingErnest` from https://stackoverflow.com/questions/47404653/pandas-0-21-0-timestamp-compatibility-issue-with-matplotlib
pd.plotting.register_matplotlib_converters()

## Acceleration, Velocity, and Position

From [Acceleration - Wikipedia](https://en.wikipedia.org/wiki/Acceleration), [Integrals - Wikipedia](https://en.wikipedia.org/wiki/Integral), and [Summation: Calculus of Finite Differences - Wikipedia](https://en.wikipedia.org/wiki/Summation#Calculus_of_finite_differences)

Acceleration can be defined as the change in velocity over time, which, in turn, is position/displacement over time $$\mathbf{\bar{a}} = \frac{\Delta \mathbf{v}}{\Delta t} = \frac{\frac{\Delta \mathbf{x}}{\Delta t}}{\Delta t} = \frac{\Delta \mathbf{x}}{\Delta t^2} $$
_Instantaneous acceleration_ is $$\mathbf{\bar{a}} = \lim_{\Delta{t}\to 0}\frac{\Delta \mathbf{v}}{\Delta t} = \frac{d\mathbf{v}}{dt} = \lim_{\Delta{t}\to 0}\frac{\Delta \mathbf{x}}{\Delta t}$$
This means we can get the velocity by taking the integral of acceleration, $$\mathbf{v} = \int_{t_0}^{t_n} \mathbf{\bar{a}} dt $$
and then position/displacement $$\mathbf{x} = \int \mathbf{v} dt = \iint_{t_0}^{t_n} \mathbf{\bar{a}} dt^2$$
making position/displacement $$\mathbf{\bar{a}}=\frac{d^{2}\mathbf{x}}{dt^2}$$

Summation $\Sigma$ is a discrete form of the continuous (by-definition) Integral $\int$. In these cases, $\Delta t$ is multiplied in order to get this sum. Since we have discrete time steps, we can use the cumulative sum instead of time.

$$
\mathbf{v}_n = \Sigma_{t=0}^{n} \mathbf{a}_{t-1}
\\[1em]
\mathbf{x}_n = \frac{(\Sigma_{t=0}^{n} \mathbf{v}_{t}) - \mathbf{v}_{n}}{2}
$$

To be honest, I don't really get where the $2$ in $\frac{\mathbf{v}_n}{2}$ comes from. I think it has to do with $$ \Delta\mathbf{\bar{v}} = \frac{\mathbf{v}_t - \mathbf{v}_{t-1}}{2} $$ but that's just a guess. It does work though! <br/><br/>Check it...


In [4]:
g = -9.807 #x/t^2
start_time = 0 
end_time = 20 #seconds
time = np.arange(start_time, end_time)
g_acceleration = pd.Series([g]*end_time)
g_velocity = g_acceleration * time
g_position = g_velocity * time/2 # t'

g_acceleration_sum = pd.concat([pd.Series([0]), g_acceleration.cumsum()], ignore_index=True) # offset by one, velocity
g_velocity_sum = g_velocity.cumsum() - g_velocity/2 # position

grav_df = pd.DataFrame({'acceleration': g_acceleration, 'velocity': g_velocity, 'position': g_position, 'acceleration_sum': g_acceleration_sum, 'velocity_sum': g_velocity_sum})
grav_df.index.name = ''
display(grav_df.iloc[:-1])

,acceleration,velocity,position,acceleration_sum,velocity_sum
,,,,,
0,-9.807,-0.000,-0.0000,0.000,0.0000
1,-9.807,-9.807,-4.9035,-9.807,-4.9035
2,-9.807,-19.614,-19.6140,-19.614,-19.6140
3,-9.807,-29.421,-44.1315,-29.421,-44.1315
4,-9.807,-39.228,-78.4560,-39.228,-78.4560
5,-9.807,-49.035,-122.5875,-49.035,-122.5875
6,-9.807,-58.842,-176.5260,-58.842,-176.5260
7,-9.807,-68.649,-240.2715,-68.649,-240.2715
8,-9.807,-78.456,-313.8240,-78.456,-313.8240


In [5]:
fig = plt.figure()
fig.canvas.layout.width = '1200px'
fig.canvas.layout.height = '1200px'

ax = fig.add_subplot(111)

for col_name, col in grav_df.items():
    
    linestyle = '-.' if '_sum' in col_name else '-'
    ax.plot(col, label=col_name, linestyle=linestyle)
    
ax.set_xlabel('Time')
ax.set_ylabel('Units')
ax.legend()
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …